In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
demo_df = pd.read_csv('demographics.csv')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
cc_df = pd.read_csv('cc.csv')
kplus_df = pd.read_csv('kplus.csv')

demo_df = demo_df.dropna()
cc_df = cc_df.dropna()
kplus_df = kplus_df.dropna()

In [25]:
cc_drop_date_df = cc_df.drop('pos_dt', axis=1)
cc_drop_date_df['cc_count'] = 1
cc_group = cc_drop_date_df.groupby(['cc_no']).agg(sum)
cc_count_amt_df = cc_group.reset_index()
cc_count_amt_df.head()

,cc_no,cc_txn_amt,cc_count
0,2,26900,8
1,4,540000,23
2,5,12000,3
3,6,32000,3
4,7,16200,11


In [43]:
demo_df.head()

,id,cc_no,gender,ocp_cd,age
0,1,1,2,9.0,5
1,1,98397,2,9.0,5
2,2,2,2,3.0,4
3,2,9740,2,3.0,4
4,3,3,2,1.0,3


In [41]:
demo_cc_df = pd.merge(demo_df, cc_count_amt_df, on='cc_no', how='left')
demo_cc_df = demo_cc_df.fillna(0)
demo_cc_df.head()

,id,cc_no,gender,ocp_cd,age,cc_txn_amt,cc_count
0,1,1,2,9.0,5,0.0,0.0
1,1,98397,2,9.0,5,44700.0,5.0
2,2,2,2,3.0,4,26900.0,8.0
3,2,9740,2,3.0,4,38300.0,34.0
4,3,3,2,1.0,3,0.0,0.0


In [45]:
demo_drop_cc_no_df = demo_cc_df.drop('cc_no', axis=1)
demo_group = demo_drop_cc_no_df.groupby(['id', 'gender', 'ocp_cd', 'age']).agg('sum')
demo_no_cc_df = demo_group.reset_index()
demo_no_cc_df.head()

,id,gender,ocp_cd,age,cc_txn_amt,cc_count
0,1,2,9.0,5,44700.0,5.0
1,2,2,3.0,4,65200.0,42.0
2,3,2,1.0,3,0.0,0.0
3,4,2,3.0,5,770000.0,36.0
4,5,2,9.0,4,12000.0,3.0


In [32]:
kplus_drop_sunday_df = kplus_df.drop('sunday', axis=1)
kplus_group = kplus_drop_sunday_df.groupby(['id']).agg('sum')
kplus_count_amt_df = kplus_group.reset_index()
kplus_count_amt_df.head()

,id,kp_txn_count,kp_txn_amt
0,1,2,3500
1,2,104,210000
2,3,70,139700
3,4,315,6076100
4,6,10,13400


In [47]:
data_df = pd.merge(demo_no_cc_df, kplus_count_amt_df, on='id', how='left')
data_df = data_df.fillna(0)
data_df

,id,gender,ocp_cd,age,cc_txn_amt,cc_count,kp_txn_count,kp_txn_amt
0,1,2,9.0,5,44700.0,5.0,2.0,3500.0
1,2,2,3.0,4,65200.0,42.0,104.0,210000.0
2,3,2,1.0,3,0.0,0.0,70.0,139700.0
3,4,2,3.0,5,770000.0,36.0,315.0,6076100.0
4,5,2,9.0,4,12000.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...
64979,64996,2,3.0,2,0.0,0.0,20.0,41200.0
64980,64997,2,3.0,2,2100.0,1.0,76.0,201200.0
64981,64998,1,9.0,3,0.0,0.0,92.0,162000.0
64982,64999,2,3.0,3,23500.0,18.0,106.0,494800.0


In [48]:
data_df.to_csv('data_df.csv', index=False)

In [59]:
import math
norm_data_df = data_df.copy()
norm_data_df['cc_txn_amt'] = norm_data_df['cc_txn_amt'].apply(lambda x: 0 if x < 10 else math.log(x, 10))
norm_data_df['kp_txn_amt'] = norm_data_df['kp_txn_amt'].apply(lambda x: 0 if x < 10 else math.log(x, 10))

In [64]:
norm_data_df.to_csv('norm_data_df.csv', index=False)
norm_data_df.head()

,id,gender,ocp_cd,age,cc_txn_amt,cc_count,kp_txn_count,kp_txn_amt
0,1,2,9.0,5,4.650308,5.0,2.0,3.544068
1,2,2,3.0,4,4.814248,42.0,104.0,5.322219
2,3,2,1.0,3,0.000000,0.0,70.0,5.145196
3,4,2,3.0,5,5.886491,36.0,315.0,6.783625
4,5,2,9.0,4,4.079181,3.0,0.0,0.000000
